In [1]:
from Scripts.twitter_auth import bearer_token
import configparser
import tweepy
import time
import pandas as pd

config = configparser.ConfigParser()
config.read("config.ini")

api_key = config["twitter"]["api_key"]
api_key_secret = config["twitter"]["api_Key_secret"]

client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

access_token = config["twitter"]["access_token"]
access_token_secret = config["twitter"]["access_Token_Secret"]

auth = tweepy.OAuth1UserHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Successful Authentication")
except:
    print("Failed authentication")

ModuleNotFoundError: No module named 'Scripts.twitter_auth'

In [7]:
tweets = []

for response in tweepy.Paginator(
    client.search_all_tweets,
    query="PTSD  #mentalhealth #veteran #Military #servicemen -is not:retweet lang:en",
    user_fields=["username", "public_metrics", "description", "location"],
    tweet_fields=["created_at", "geo", "public_metrics", "text"],
    expansions="author_id",
    start_time="2006-10-01T00:00:00Z",
    end_time="2022-11-17T00:00:00Z",
    max_results=500,
    limit=250000,
):
    time.sleep(1)
    tweets.append(response)


Rate limit exceeded. Sleeping for 194 seconds.


KeyboardInterrupt: 

In [8]:
result = []
user_dict = {}
# Loop through each response object
for response in tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes["users"]:
        user_dict[user.id] = {
            "username": user.username,
            "followers": user.public_metrics["followers_count"],
            "tweets": user.public_metrics["tweet_count"],
            "description": user.description,
            "location": user.location,
        }
    for tweet in response.data:
        hashtags = []
        try:
            for hashtag in tweet.entities["hashtags"]:
                hashtags.append(hashtag["text"])
        except:
            pass
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append(
            {
                "author_id": tweet.author_id,
                "username": author_info["username"],
                "author_followers": author_info["followers"],
                "author_tweets": author_info["tweets"],
                "author_description": author_info["description"],
                "author_location": author_info["location"],
                "text": tweet.text,
                "created_at": tweet.created_at,
                "date": tweet.created_at,
                "hashtags": [hashtags if hashtags else None],
                "source": tweet.source,
                "retweets": tweet.public_metrics["retweet_count"],
                "replies": tweet.public_metrics["reply_count"],
                "likes": tweet.public_metrics["like_count"],
                "quote_count": tweet.public_metrics["quote_count"],
            }
        )

# Change this list of dictionaries into a dataframe
df_tweets = pd.DataFrame(result)


In [10]:
df_tweets.to_csv("../Data/all_tweets.csv", index=False)
df_tweets = pd.read_csv("all_tweets.csv")
df_tweets.shape

(198228, 15)

In [11]:
df_tweets.head()

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,date,hashtags,source,retweets,replies,likes,quote_count
0,382707440,GraniteStateCol,965,3699,Focused on helping adult students achieve pers...,"New Hampshire, US","As an adult student, Courtney balanced her edu...",2022-11-16 23:59:59+00:00,2022-11-16 23:59:59+00:00,[None],NaN,0,0,0,0
1,83999520,hferre,352,47026,"Humanista, pro de la igualdad, al derecho de t...",NaN,RT @ggreenwald: US funding for war in Ukraine ...,2022-11-16 23:59:58+00:00,2022-11-16 23:59:58+00:00,[None],NaN,13160,0,0,0
2,2836578730,Nuages1862,242,53738,“The colour of my soul is iron-grey and sad ba...,NaN,RT @TulsiGabbard: Biden Admin is giving blank ...,2022-11-16 23:59:56+00:00,2022-11-16 23:59:56+00:00,[None],NaN,4470,0,0,0
3,1578424436528623627,MariaJo10278724,140,2310,we hope doing well in exam,NaN,Enjoy defense sure what cup country network an...,2022-11-16 23:59:56+00:00,2022-11-16 23:59:56+00:00,[None],NaN,0,0,0,0
4,558490660,yogeshjoshi603,144,2087,Nation First…🇮🇳🇮🇳🇮🇳Live Life KingSize....,India,RT @ANI: #WATCH | Lance Naik Manju of Eastern ...,2022-11-16 23:59:55+00:00,2022-11-16 23:59:55+00:00,[None],NaN,131,0,0,0


In [13]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198228 entries, 0 to 198227
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   author_id           198228 non-null  int64  
 1   username            198228 non-null  object 
 2   author_followers    198228 non-null  int64  
 3   author_tweets       198228 non-null  int64  
 4   author_description  154187 non-null  object 
 5   author_location     108892 non-null  object 
 6   text                198228 non-null  object 
 7   created_at          198228 non-null  object 
 8   date                198228 non-null  object 
 9   hashtags            198228 non-null  object 
 10  source              0 non-null       float64
 11  retweets            198228 non-null  int64  
 12  replies             198228 non-null  int64  
 13  likes               198228 non-null  int64  
 14  quote_count         198228 non-null  int64  
dtypes: float64(1), int64(7), object(7)